Flask on Google Colab

https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b


In [ ]:
!pip install flask-ngrok

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
#montagem padrao
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#copiar checkpooint do drive para area de trabalho do colab
!cp -r '/content/drive/My Drive/final_model_VGG16.h5' '/content/'

In [ ]:
#copiar checkpooint do drive para area de trabalho do colab
!cp -r '/content/drive/My Drive/final_model_Xception.h5' '/content/'

In [ ]:
#copiar flask static files do drive para area de trabalho do colab
!cp -r '/content/drive/My Drive/flask_files/static' '/content/'

In [ ]:
#copiar flask templates files do drive para area de trabalho do colab
!cp -r '/content/drive/My Drive/flask_files/templates' '/content/'

In [ ]:
#copiar checkpoints para area de trabalho do colab
!cp -r '/content/drive/My Drive/checkpoint' '/content/'

In [ ]:
#escolhendo o modelo
MODELO = '/content/final_model_Xception.h5'

# Testando o Modelo

In [ ]:
#@title
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.models import model_from_json

modeljson = '/content/checkpoint/model_X.json'
modeljson_weights = '/content/checkpoint/model_X_weights.h5'


# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('/content/test_dog.jpg')
	# load model
	#model = load_model('/content/final_model_VGG16.h5')
	#model = load_model('/content/final_model_Xception.h5')

	# load json and create model
	with open(modeljson, 'r') as json_file:
		loaded_model_json = json_file.read()

	model = model_from_json(loaded_model_json)
	# load weights into new model
	model.load_weights(modeljson_weights)

	# predict the class
	result = model.predict(img)
	print(result)
	if result[0]>0.5:
		print( " is a dog")
	else:
		print( " is a cat")


In [ ]:
#@title
# entry point, run the example
run_example()

[[0.99939466]]
 is a dog


# Running Flask CATDOG Model

In [ ]:
#@title Flask Code
# USAGE
# Start the server:
# 	python run_keras_server.py
# Submit a request via cURL:
# 	curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'
# Submita a request via Python:
#	python simple_request.py

# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
from flask import Flask, request, render_template, jsonify, abort, url_for, redirect
import io
from keras.models import load_model
#from keras.applications import VGG16
from flask_ngrok import run_with_ngrok
from keras.models import model_from_json

# initialize our Flask application and the Keras model
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
model = None
#FINAL_MODEL_VGG16 = MODELO
modeljson = '/content/checkpoint/model_X.json'
modeljson_weights = '/content/checkpoint/model_X_weights.h5'


def load_modelo():
	# load the pre-trained Keras model (here we are using a model
	# pre-trained on ImageNet and provided by Keras, but you can
	# substitute in your own networks just as easily)
	global model
	# load model
	#model = load_model(FINAL_MODEL_VGG16)
	# load json and create model
	with open(modeljson, 'r') as json_file:
		loaded_model_json = json_file.read()

	model = model_from_json(loaded_model_json)
	# load weights into new model
	model.load_weights(modeljson_weights)


def prepare_image(image, target):
	# if the image mode is not RGB, convert it
	if image.mode != "RGB":
		image = image.convert("RGB")

	# resize the input image and preprocess it
	image = image.resize(target)
	image = img_to_array(image)
	image = np.expand_dims(image, axis=0)
	image = imagenet_utils.preprocess_input(image)

	# return the processed image
	return image


def its_cat_or_dog(predict):
  print(predict[0])
  if predict[0]>0.5:
    return ( " is a dog")
  else:
    return  " is a cat"


# @app.route("/")
# def hello():
#     return "Hello World!"

@app.route("/")
@app.route("/base")
def base():
    return render_template('dashboard.html')


@app.route("/predict", methods=["POST"])
def predict():
	# initialize the data dictionary that will be returned from the
	# view
	data = {"success": False}

	# ensure an image was properly uploaded to our endpoint
	if request.method == "POST":
		if request.files.get("image"):
			# read the image in PIL format
			image = request.files["image"].read()
			image = Image.open(io.BytesIO(image))

			# preprocess the image and prepare it for classification
			image = prepare_image(image, target=(224, 224))

			# classify the input image and then initialize the list
			# of predictions to return to the client
			preds = model.predict(image)
			results = its_cat_or_dog(preds)
   
			data["predictions"] = []

			# loop over the results and add them to the list of
			# returned predictions
			r = {"label": results, "probability": float(preds[0])}
			data["predictions"].append(r)

			# indicate that the request was a success
			data["success"] = True

	# return the data dictionary as a JSON response
	return jsonify(data)


# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
	print(("* Loading Keras model and Flask starting server..."
            "please wait until server has fully started"))
	load_modelo()
	app.run()


* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2514c0003f84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Oct/2020 03:13:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 03:13:12] "GET /static/image/header_foto.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 03:13:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2020 03:13:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 03:13:34] "POST /predict HTTP/1.1" 200 -


[1.2909078e-05]


127.0.0.1 - - [28/Oct/2020 03:13:45] "POST /predict HTTP/1.1" 200 -


[0.9993746]


# Nova seção

In [ ]:
model.save_weights("Xcept_weights.hs")

In [ ]:
from pathlib import Path
Path('/content/Xcept_weights.hs.index').stat().st_size

In [ ]:
#load weights
model.load_weights("weights.best.hdf5")
#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[ 'accuracy'])
print("Created model and loaded weights from file")

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model_X.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_X_weights.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
#copiar checkpoints para area de trabalho do colab
!cp -r '/content/drive/My Drive/checkpoint' '/content/'

In [ ]:
modeljson = '/content/checkpoint/model_X.json'
modeljson_weights = '/content/checkpoint/model_X_weights.h5'
# load json and create model
json_file = open(modeljson, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(modeljson_weights)
print("Loaded model from disk")

Loaded model from disk


In [ ]:
print(loaded_model)

In [ ]:
loaded_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_22  (None, 2048)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_44 (Dense)             (None, 128)               262272    
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 129       
Total params: 21,123,881
Trainable params: 21,069,353
Non-trainable params: 54,528
_________________________________________________________________
